In [2]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
#These are mainly for opening objects 
import os
import sys
import time
import pickle
import gzip
import numpy as np
import h5py

#The real guts of the Deep learning
import theano
import theano.tensor as T

import lasagne
from lasagne import layers
from lasagne.updates import nesterov_momentum
from lasagne.objectives import squared_error
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import BatchIterator

In [ ]:
#Things you can change
os.environ['LASAGNE_DATA_PATH'] = '/notebooks/ml-sims/data/nanoparticle/'
numPart = 1000       #number of particles I will be tracking

In [ ]:
#Now, let's load the particles

totalPart = 262144   #total number of particles

trainingFrac = 0.8
validFrac = 0.1

#Here, we are selecting random particles to spy on
np.random.seed(0)
indexes = np.random.choice(totPart, numPart)

slice = np.s_[idxs, :]

X = np.zeros((100, numPart*6))
y = np.zeros((100, numPart*3))

colors = ['b','g', 'r', 'y', 'k', 'm']

absMinVel, absMaxVal = 0,0
maxCoord = 10000

for i in xrange(101):
    fname = getFilename(i)
    f = h5py.File(fname, 'r')
    ids = f['PartType1']['ParticleIDs'][()]
    sorter = ids.argsort() 
    
    coords = f['PartType1']['Coordinates'][()] 
    coords = coords[sorter]#sort by ids
    
    #normalize coordinates (just divide by max)
    coords/=maxCoord 

    #Here we test so see how the coordinates loaded
    #from matplotlib import pyplot as plt
    #plt.scatter(coords[0, 0], coords[0,1], c = colors[i%len(colors)]) 

    coords = coords[slice]
    
    if i!=0: 
        y[i-1,:] = coords.flatten() 
    if i!=100:
        vels = f['PartType1']['Velocities'][()] 
        vels = vels[sorter]
        
        minVel, maxVel = vels.min(), vels.max()
        if minVel < absMinVel:
            absMinVel = minVel
           
        if maxVel > absMaxVal:
            absMaxVal = maxVel

        vels = vels[slice] 
        data = np.concatenate((coords, vels), axis = 1).flatten() 
                                                                                                
        X[i,:] = data
        del data     #Cleaning out the variables for next time      

    del coords      #Still cleaning
    f.close()       #Closing the file!

    #Normalizing the Velocity coords
    for n in xrange(nParticles):
        for col in xrange(3):
            X[:, n*6+3+col] = (X[:, n*6+3+col]-absMinVel)/(absMaxVal-absMinVel)

In [ ]:
#loading in the particle files
def getFilename(i):          
    base = path+'snapshot_' 
    if i<10:                 
        out= base+'00%d.hdf5'%i
        elif i<100:        
        out= base+'0%d.hdf5'%i 
    else:           
        out= base+'%d.hdf5'%i 
    return path+out       

In [ ]:
def renormalize(array):
    return (array - array.min()) / (array.max() - array.min())

for i in range(5):
    X[:, i, :, :] = renormalize(X[:, i, :, :])
    
y = renormalize(y)

print("X.shape = {}, X.min = {}, X.max = {}".format(X.shape, X.min(), X.max()))
print("y.shape = {}, y.min = {}, y.max = {}".format(y.shape, y.min(), y.max()))

In [ ]:
def compute_PCA(array):

    nimages0, nchannels0, height0, width0 = array.shape
    rolled = np.transpose(array, (0, 2, 3, 1))
    # transpose from N x channels x height x width  to  N x height x width x channels
    nimages1, height1, width1, nchannels1 = rolled.shape
    # check shapes
    assert nimages0 == nimages1
    assert nchannels0 == nchannels1
    assert height0 == height1
    assert width0 == width1
    # flatten
    reshaped = rolled.reshape(nimages1 * height1 * width1, nchannels1)
    
    from sklearn.decomposition import PCA
    
    pca = PCA()
    pca.fit(reshaped)
    
    cov = pca.get_covariance()
    
    eigenvalues, eigenvectors = np.linalg.eig(cov)
    
    return eigenvalues, eigenvectors

In [ ]:
class AugmentedBatchIterator(BatchIterator):
    
    def __init__(self, batch_size, crop_size=8, testing=False):
        super(AugmentedBatchIterator, self).__init__(batch_size)
        self.crop_size = crop_size
        self.testing = testing

    def transform(self, Xb, yb):

        Xb, yb = super(AugmentedBatchIterator, self).transform(Xb, yb)
        batch_size, nchannels, width, height = Xb.shape
        
        if self.testing:
            if self.crop_size % 2 == 0:
                right = left = self.crop_size // 2
            else:
                right = self.crop_size // 2
                left = self.crop_size // 2 + 1
            X_new = Xb[:, :, right: -left, right: -left]
            return X_new, yb

        eigenvalues, eigenvectors = compute_PCA(Xb)

        # Flip half of the images horizontally at random
        indices = np.random.choice(batch_size, batch_size // 2, replace=False)        
        Xb[indices] = Xb[indices, :, :, ::-1]

        # Crop images
        X_new = np.zeros(
            (batch_size, nchannels, width - self.crop_size, height - self.crop_size),
            dtype=np.float32
        )

        for i in range(batch_size):
            # Choose x, y pixel posiitions at random
            px, py = np.random.choice(self.crop_size, size=2)
                
            sx = slice(px, px + width - self.crop_size)
            sy = slice(py, py + height - self.crop_size)
            
            # Rotate 0, 90, 180, or 270 degrees at random
            nrotate = np.random.choice(4)
            
            # add random color perturbation
            alpha = np.random.normal(loc=0.0, scale=0.5, size=5)
            noise = np.dot(eigenvectors, np.transpose(alpha * eigenvalues))
            
            for j in range(nchannels):
                X_new[i, j] = np.rot90(Xb[i, j, sx, sy] + noise[j], k=nrotate)
                
        return X_new, yb

In [ ]:
class SaveParams(object):
    def __init__(self, name):
        self.name = name

    def __call__(self, nn, train_history):
        if train_history[-1]["valid_loss_best"]:
            nn.save_params_to("{}.params".format(self.name))
            with open("{}.history".format(self.name), "w") as f:
                pickle.dump(train_history, f)

In [ ]:
net49 = NeuralNet(
    layers=[
        ('input', layers.InputLayer),

        ('conv11', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),

        ('conv21', layers.Conv2DLayer),
        ('conv22', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),

        ('conv31', layers.Conv2DLayer),
        ('conv32', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),

        ('hidden4', layers.DenseLayer),
        ('dropout4', layers.DropoutLayer),
        
        ('hidden5', layers.DenseLayer),
        ('dropout5', layers.DropoutLayer),

        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 5, 44, 44),
    
    conv11_num_filters=32, conv11_filter_size=(5, 5), 
    pool1_pool_size=(2, 2),

    conv21_num_filters=64, conv21_filter_size=(3, 3),
    conv22_num_filters=64, conv22_filter_size=(3, 3),
    pool2_pool_size=(2, 2),

    conv31_num_filters=128, conv31_filter_size=(3, 3),
    conv32_num_filters=128, conv32_filter_size=(3, 3),
    pool3_pool_size=(2, 2),

    hidden4_num_units=2048,
    dropout4_p=0.5,
    
    hidden5_num_units=2048,
    dropout5_p=0.5,

    output_num_units=1,
    output_nonlinearity=None,

    update_learning_rate=0.0001,
    update_momentum=0.9,

    objective_loss_function=squared_error,
    regression=True,
    max_epochs=1000,
    batch_iterator_train=AugmentedBatchIterator(batch_size=128, crop_size=4),
    batch_iterator_test=AugmentedBatchIterator(batch_size=128, crop_size=4, testing=True),

    on_epoch_finished=[SaveParams("net49")],

    verbose=2,
    )